In [1]:
import requests
from bs4 import BeautifulSoup
import bs4
import json
import httpx
import time
import lxml
import cohere
from cohere.classify import Example 
import pandas as pd
from unidecode import unidecode

In [2]:
cohere_apiKey = '6rIWhaa4vGVQKRnSJ9ClSKkgsxQp8EabiXBLH2zl'
elsevier_apikey = 'ce04ffa20c7ee4560e4ffd4761bbe157'

# Papaer Parser Functions

In [28]:
class paper_parser():
    def __init__(self, query, apikey, count, start, sort, view, timeout):
        self.query = query
        self.apikey = apikey
        self.count = count
        self.start = start
        self.sort = sort
        self.view = view
        self.timeout = timeout
        self.dic = self.search_for_papers(self.query, self.apikey, self.count, self.start, self.sort, self.view)
        self.piis = list(self.dic.values())
        self.paragraphs = self.get_all_paragraphs()
        self.titles = self.get_all_titles()
        self.authors = self.get_all_1st_authors()
        self.organizations = self.get_all_organizations()
    def search_for_papers(self, query, apikey, count, start, sort, view):
        url = 'https://api.elsevier.com/content/search/sciencedirect?'
        headers = {'X-ELS-APIKey': apikey, 'Accept': 'application/json'}
        params = {'query': query, 'count': count, 'start': start, 'sort': sort, 'view': view}
        r = requests.get(url, headers=headers, params=params)
        json_resp = json.loads(r.text)
        titles, piis, authors = [], [], []
        dic = {}
        for paper in json_resp['search-results']['entry']:
            titles.append(paper['dc:title'])
            piis.append(paper['pii'])
            dic[paper['dc:title']] = paper['pii']
        return dic
    def get_paragraphs(self, pii):
        url = f'https://api.elsevier.com/content/article/pii/{pii}'
        response = requests.get(url, headers={"X-ELS-APIKey":self.apikey, "content-type": "text/xml;charset=UTF-8"}, timeout = self.timeout)
        soup = BeautifulSoup(response.content, "xml")
        text =[unidecode(''.join(s.findAll(text=True)).replace('\n', '')) for s in soup.find_all('ce:para')]
        return text
    def get_all_1st_authors(self):
        authors = []
        for pii in self.piis:
            url = f'https://api.elsevier.com/content/article/pii/{pii}'
            response = requests.get(url, headers={"X-ELS-APIKey":self.apikey, "content-type": "text/xml;charset=UTF-8"}, timeout = self.timeout)
            soup = BeautifulSoup(response.content, "xml")
            first_name = [''.join(s.findAll(text=True)) for s in soup.find_all('ce:given-name')][0]
            last_name = [''.join(s.findAll(text=True)) for s in soup.find_all('ce:surname')][0]
            name = first_name + ' ' + last_name
            authors.append(name)
        return authors
    def get_all_organizations(self):
        organizations = []
        for pii in self.piis:
            url = f'https://api.elsevier.com/content/article/pii/{pii}'
            response = requests.get(url, headers={"X-ELS-APIKey":self.apikey, "content-type": "text/xml;charset=UTF-8"}, timeout = self.timeout)
            soup = BeautifulSoup(response.content, "xml")            
            organization = [''.join(s.findAll(text=True)) for s in soup.find_all('sa:organization')]
            # join all organizations into one string and split by comma
            organization = ', '.join(organization)
            organizations.append(organization)
        return organizations
    def get_all_paragraphs(self):
        paragraphs = []
        for pii in self.piis:
            paragraphs.append(self.get_paragraphs(pii))
        return paragraphs
    def get_all_titles(self):
        return list(self.dic.keys())
    def get_text(self, pii, apikey, timeout):
        url = f'https://api.elsevier.com/content/article/pii/{pii}'
        response = requests.get(url, headers={"X-ELS-APIKey":apikey, "content-type": "text/xml;charset=UTF-8"}, timeout = timeout)
        soup = BeautifulSoup(response.content, "xml")
        return soup

# Parsing papers for examples

In [29]:
pp = paper_parser('nips, membrane, polysulfone', elsevier_apikey, 10, 0, 'relevance', 'COMPLETE', 60)
example_papers = pp.get_all_paragraphs()
example_titles = pp.get_all_titles()
example_authors = pp.get_all_1st_authors()
example_organizations = pp.get_all_organizations()

In [22]:
pp.piis

['S037673882200343X',
 'S1876107021005460',
 'S2213343722016402',
 'S037673882200881X',
 'S1383586621007437',
 'S1383586622011091',
 'S0927775721016113',
 'S1385894722040748',
 'S1876107022004072',
 'S0376738820312667']

In [30]:
example_organizations

['R&D Center for Membrane Technology, Chung Yuan Christian University, Department of Chemical Engineering, Chung Yuan Christian University, Department of Chemical Engineering, National Taiwan University, R&D Center for Membrane Technology, Chung Yuan Christian University',
 'State Key Laboratory of Separation Membranes and Membrane Processes, School of Material Science and Engineering, Tiangong University',
 'School of Chemistry, College of Science, University of Tehran',
 'Center for Membrane Separation and Water Science & Technology, College of Chemical Engineering, Zhejiang University of Technology, College of Chemistry and Materials Engineering, Wenzhou University, Center for Membrane Separation and Water Science & Technology, College of Chemical Engineering, Zhejiang University of Technology',
 'School of Chemistry, College of Science, University of Tehran',
 'Beijing International Scientific and Technological Cooperation Base of Water Pollution Control Techniques for Antibiotics 

In [31]:
pp.get_text(pp.piis[0], elsevier_apikey, 60)

<?xml version="1.0" encoding="utf-8"?>
<full-text-retrieval-response xmlns="http://www.elsevier.com/xml/svapi/article/dtd" xmlns:bk="http://www.elsevier.com/xml/bk/dtd" xmlns:cals="http://www.elsevier.com/xml/common/cals/dtd" xmlns:ce="http://www.elsevier.com/xml/common/dtd" xmlns:dc="http://purl.org/dc/elements/1.1/" xmlns:dcterms="http://purl.org/dc/terms/" xmlns:ja="http://www.elsevier.com/xml/ja/dtd" xmlns:mml="http://www.w3.org/1998/Math/MathML" xmlns:prism="http://prismstandard.org/namespaces/basic/2.0/" xmlns:sa="http://www.elsevier.com/xml/common/struct-aff/dtd" xmlns:sb="http://www.elsevier.com/xml/common/struct-bib/dtd" xmlns:tb="http://www.elsevier.com/xml/common/table/dtd" xmlns:xlink="http://www.w3.org/1999/xlink" xmlns:xocs="http://www.elsevier.com/xml/xocs/dtd" xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance"><coredata><prism:url>https://api.elsevier.com/content/article/pii/S037673882200343X</prism:url><dc:identifier>doi:10.1016/j.memsci.2022.120597</dc:identifier><

In [6]:
example_papers[0]

['Versatile, facile, easy, and economical, these are the advantages that immersion precipitation or non-solvent phase separation (NIPS) offers [1-3]. Due to these, NIPS process has been regarded as the most widely-used approach in fabricating flat sheet membranes from a variety of polymers [4]. In addition, NIPS can provide a wide-range of pore size: from microfiltration to nanofiltration [5]. In this process, a polymer of choice is dissolved in a suitable solvent. Once homogeneous, the polymeric solution is cast onto a support and phase-inversion is prompted through direct immersion in a coagulation bath/non-solvent bath (typically water), generating a polymer rich phase that transforms into the membrane matrix after polymer precipitation, and a polymer lean phase which develops into pores after demixing [6].',
 'Polysulfone (PSf) membrane is amongst the most extensively used commercial membranes due to its favorable attributes such as high glass transition temperature (Tg = 190 degC)

In [32]:
example_titles

['Strategy to prepare skin-free and macrovoid-free polysulfone membranes via the NIPS process',
 'Towards high-performance polysulfone membranes: A controllable membrane formation process using surfactant in NIPS',
 'Efficient dye rejection using a mixed matrix polyphenylsulfone/polysulfone membrane containing basil seed mucilage hydrogel',
 'Tight UF membranes with ultrahigh water flux prepared by <ce:italic>in-situ</ce:italic> growing ZIF particles in NIPS process for greatly enhanced dye removal efficiency',
 'Preparation of mixed matrix membranes made up of polysulfone and MIL-53(Al) nanoparticles as promising membranes for separation of aqueous dye solutions',
 'Fabrication of polysulfone membrane with sponge-like structure by using different non-woven fabrics',
 'Development of polysulfone ultrafiltration membranes with enhanced antifouling performance for the valorisation of side streams in the pulp and paper industry',
 'Photocatalytic polysulfone membrane incorporated by ZnO-M

In [55]:
def flatten_list(pp):
    paragraphs, titles, piis, authors, organizations = [], [], [], [], []
    _2d_list = pp.paragraphs
    # Iterate through the outer list
    for i in range(len(_2d_list)):
        if type(_2d_list[i]) is list:
            # If the element is of type list, iterate through the sublist
            for item in _2d_list[i]:
                paragraphs.append(item)
                titles.append(pp.titles[i])
                piis.append(pp.piis[i])
                authors.append(pp.authors[i])
                organizations.append(pp.organizations[i])
        else:
            paragraphs.append(_2d_list[i])
    return [paragraphs, titles, piis, authors, organizations]

In [58]:
flatten_list(pp)

[['Versatile, facile, easy, and economical, these are the advantages that immersion precipitation or non-solvent phase separation (NIPS) offers [1-3]. Due to these, NIPS process has been regarded as the most widely-used approach in fabricating flat sheet membranes from a variety of polymers [4]. In addition, NIPS can provide a wide-range of pore size: from microfiltration to nanofiltration [5]. In this process, a polymer of choice is dissolved in a suitable solvent. Once homogeneous, the polymeric solution is cast onto a support and phase-inversion is prompted through direct immersion in a coagulation bath/non-solvent bath (typically water), generating a polymer rich phase that transforms into the membrane matrix after polymer precipitation, and a polymer lean phase which develops into pores after demixing [6].',
  'Polysulfone (PSf) membrane is amongst the most extensively used commercial membranes due to its favorable attributes such as high glass transition temperature (Tg = 190 deg

In [64]:
Example_data = pd.DataFrame(flatten_list(pp)).transpose()
# add column names
Example_data.columns = ['paragraph', 'title', 'pii', 'author', 'organization']
#Example_data.to_csv('Example_data_info.csv')
Example_data

,paragraph,title,pii,author,organization
0,"Versatile, facile, easy, and economical, these...",Strategy to prepare skin-free and macrovoid-fr...,S037673882200343X,Irish Valerie Maggay,"R&D Center for Membrane Technology, Chung Yuan..."
1,Polysulfone (PSf) membrane is amongst the most...,Strategy to prepare skin-free and macrovoid-fr...,S037673882200343X,Irish Valerie Maggay,"R&D Center for Membrane Technology, Chung Yuan..."
2,As a result of the raising interest to improve...,Strategy to prepare skin-free and macrovoid-fr...,S037673882200343X,Irish Valerie Maggay,"R&D Center for Membrane Technology, Chung Yuan..."
3,Standard NIPS process yield skinned asymmetric...,Strategy to prepare skin-free and macrovoid-fr...,S037673882200343X,Irish Valerie Maggay,"R&D Center for Membrane Technology, Chung Yuan..."
4,Although adding solvent to the coagulation bat...,Strategy to prepare skin-free and macrovoid-fr...,S037673882200343X,Irish Valerie Maggay,"R&D Center for Membrane Technology, Chung Yuan..."
...,...,...,...,...,...
464,"Dinglei Zhong: Investigation, Data curation, W...",Additive-free preparation of hemodialysis memb...,S0376738820312667,Dinglei Zhong,State Key Laboratory of Materials-Oriented Che...
465,The authors declare the following financial in...,Additive-free preparation of hemodialysis memb...,S0376738820312667,Dinglei Zhong,State Key Laboratory of Materials-Oriented Che...
466,Financial support from the National Natural Sc...,Additive-free preparation of hemodialysis memb...,S0376738820312667,Dinglei Zhong,State Key Laboratory of Materials-Oriented Che...
467,The following is the Supplementary data to thi...,Additive-free preparation of hemodialysis memb...,S0376738820312667,Dinglei Zhong,State Key Laboratory of Materials-Oriented Che...


# Training the classifier

In [73]:
example_df = pd.read_csv('Example_data_labeled_updated.csv')
example_df
#count the number of y in label
print(example_df['label'].value_counts())
example_df

n    456
y     31
Name: label, dtype: int64


,paragraph,label
0,"Versatile, facile, easy, and economical, these...",n
1,Polysulfone (PSf) membrane is amongst the most...,n
2,As a result of the raising interest to improve...,n
3,Standard NIPS process yield skinned asymmetric...,n
4,Although adding solvent to the coagulation bat...,n
...,...,...
482,"Yi Yu: Conceptualization, Methodology, Investi...",n
483,The authors declare that they have no known co...,n
484,This work is financially supported by National...,n
485,The following is the Supplementary data to thi...,n


In [69]:
def drop_long_para(df, word_count = 300):
    df_new = df.copy()
    df_new['word_count'] = df_new['paragraph'].apply(lambda x: len(x.split()))
    df_new = df_new[df_new['word_count'] < word_count]
    #df_new = df_new.drop(columns = ['word_count'])
    df_new = df_new.reset_index(drop = True)
    return df_new

# write a function to split the paragraphs into smaller ones and join by word and save the rest to the next row and keep the label
def split_para(df, word_count = 300):
    df_new = df.copy()
    df_new['word_count'] = df_new['paragraph'].apply(lambda x: len(x.split()))
    df_new['split'] = df_new['paragraph'].apply(lambda x: x.split())
    df_new['split'] = df_new['split'].apply(lambda x: [x[i:i+word_count] for i in range(0, len(x), word_count)])
    df_new['split'] = df_new['split'].apply(lambda x: [' '.join(i) for i in x])
    df_new = df_new.explode('split')
    df_new = df_new.reset_index(drop = True)
    return df_new

# define a function to shorten the paragraph to a certain length
def shorten_para(df, word_count = 300):
    df_new = df.copy()
    df_new['word_count'] = df_new['paragraph'].apply(lambda x: len(x.split()))
    df_new['split'] = df_new['paragraph'].apply(lambda x: x.split())
    df_new['split'] = df_new['split'].apply(lambda x: x[:word_count])
    df_new['split'] = df_new['split'].apply(lambda x: ' '.join(x))
    return df_new

In [74]:
# Drop the long paragraphs
example_df = shorten_para(example_df, 250)
print(example_df['label'].value_counts())
example_df

n    456
y     31
Name: label, dtype: int64


,paragraph,label,word_count,split
0,"Versatile, facile, easy, and economical, these...",n,121,"Versatile, facile, easy, and economical, these..."
1,Polysulfone (PSf) membrane is amongst the most...,n,299,Polysulfone (PSf) membrane is amongst the most...
2,As a result of the raising interest to improve...,n,106,As a result of the raising interest to improve...
3,Standard NIPS process yield skinned asymmetric...,n,264,Standard NIPS process yield skinned asymmetric...
4,Although adding solvent to the coagulation bat...,n,217,Although adding solvent to the coagulation bat...
...,...,...,...,...
482,"Yi Yu: Conceptualization, Methodology, Investi...",n,39,"Yi Yu: Conceptualization, Methodology, Investi..."
483,The authors declare that they have no known co...,n,26,The authors declare that they have no known co...
484,This work is financially supported by National...,n,49,This work is financially supported by National...
485,The following is the Supplementary data to thi...,n,13,The following is the Supplementary data to thi...


In [55]:
# get the split from example_df with label y
example_df_y = example_df[example_df['label'] == 'y']
# create a new column called dictionary
example_df_y['dictionary'] = '{"polymer":, "solvent":, "additives":, "polymer concentration":, "amount of polymer", "amount of solvent", "synthesis temperature", "synthesis time":}'
# save example_df_y to csv
example_df_y.to_csv('example_df_y.csv')

C:\Users\wang9\AppData\Roaming\Python\Python37\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


In [65]:
# print all the paragraphs with label y
for i in example_df_y['paragraph']:
    print(i)
    print('---------------------------')

The polymer polysulfone (PSf, Udel(r) P-3500, Mw = 50,800 Da, Mn = 33,500 Da, r = 1.24 g/cm3) was acquired from the Amoco Performance Products Inc. (Ridgefield, CT, USA). The solvents N-methyl-2-pyrrolidinone (NMP, r = 1.03 g/cm3), dimethylacetamide (DMAc, r = 0.940 g/cm3) and dimethylformamide (DMF, 0.944 g/cm3) of reagent grades were purchased from Uni-Onward Corp., Taiwan, while reagent grade 2-pyrrolidinone (2P, r = 1.12 g/cm3) was procured from Acros Organics. All the chemicals were used without further purification. DI water was produced in our lab using an Elga Purelab(r) water purification system.
---------------------------
A typical casting solution was prepared by dissolving PSf polymer in 2P solvent. To prepare the pristine casting solutions, 8, 10, 12 and 15 wt% of PSf polymer were added into the 2P solvent. Meanwhile, to prepare the modified casting solutions, respective amount (wt%) of non-solvent additive (water) was added into the bottle containing 2P, and mixed well f

In [171]:
example_joined = pd.read_csv('example_data_labeled_joined.csv')
for i in example_joined['paragraph']:
    print(i)
    print('---------------------------')

The polymer polysulfone (PSf, Udel(r) P-3500, Mw = 50,800 Da, Mn = 33,500 Da, r = 1.24 g/cm3) was acquired from the Amoco Performance Products Inc. (Ridgefield, CT, USA). The solvents N-methyl-2-pyrrolidinone (NMP, r = 1.03 g/cm3), dimethylacetamide (DMAc, r = 0.940 g/cm3) and dimethylformamide (DMF, 0.944 g/cm3) of reagent grades were purchased from Uni-Onward Corp., Taiwan, while reagent grade 2-pyrrolidinone (2P, r = 1.12 g/cm3) was procured from Acros Organics. All the chemicals were used without further purification. DI water was produced in our lab using an Elga Purelab(r) water purification system. A typical casting solution was prepared by dissolving PSf polymer in 2P solvent. To prepare the pristine casting solutions, 8, 10, 12 and 15 wt% of PSf polymer were added into the 2P solvent. Meanwhile, to prepare the modified casting solutions, respective amount (wt%) of non-solvent additive (water) was added into the bottle containing 2P, and mixed well for a couple of minutes. Foll

In [66]:
papers = paper_parser('nips, membrane, polysulfone', elsevier_apikey, 50, 0, 'relevance', 'COMPLETE', 60)

In [93]:
test_papers = papers.paragraphs

In [94]:
len(test_papers)


50

In [95]:
test_papers = papers.paragraphs

test_papers = test_papers[10:50]
test_papers = pd.DataFrame(flatten_list(papers)).transpose()
# add column names
test_papers.columns = ['paragraph', 'title', 'pii', 'author', 'organization']

In [96]:
test_papers

,paragraph,title,pii,author,organization
0,"Versatile, facile, easy, and economical, these...",Strategy to prepare skin-free and macrovoid-fr...,S037673882200343X,Irish Valerie Maggay,"R&D Center for Membrane Technology, Chung Yuan..."
1,Polysulfone (PSf) membrane is amongst the most...,Strategy to prepare skin-free and macrovoid-fr...,S037673882200343X,Irish Valerie Maggay,"R&D Center for Membrane Technology, Chung Yuan..."
2,As a result of the raising interest to improve...,Strategy to prepare skin-free and macrovoid-fr...,S037673882200343X,Irish Valerie Maggay,"R&D Center for Membrane Technology, Chung Yuan..."
3,Standard NIPS process yield skinned asymmetric...,Strategy to prepare skin-free and macrovoid-fr...,S037673882200343X,Irish Valerie Maggay,"R&D Center for Membrane Technology, Chung Yuan..."
4,Although adding solvent to the coagulation bat...,Strategy to prepare skin-free and macrovoid-fr...,S037673882200343X,Irish Valerie Maggay,"R&D Center for Membrane Technology, Chung Yuan..."
...,...,...,...,...,...
2114,It was observed that at a higher b-CD concentr...,Tight ultrafiltration: Layer deposition of Tri...,S2213343720300816,John Michael Tesha,State Key Laboratory of Separation Membranes a...
2115,Tight UF membranes were successfully prepared ...,Tight ultrafiltration: Layer deposition of Tri...,S2213343720300816,John Michael Tesha,State Key Laboratory of Separation Membranes a...
2116,John Michael Tesha: Writing - review & editing...,Tight ultrafiltration: Layer deposition of Tri...,S2213343720300816,John Michael Tesha,State Key Laboratory of Separation Membranes a...
2117,The authors declare that they have no known co...,Tight ultrafiltration: Layer deposition of Tri...,S2213343720300816,John Michael Tesha,State Key Laboratory of Separation Membranes a...


In [101]:
test_papers = shorten_para(test_papers,150)
test_papers.to_csv('test_data.csv')
test_papers

,paragraph,title,pii,author,organization,word_count,split
0,"Versatile, facile, easy, and economical, these...",Strategy to prepare skin-free and macrovoid-fr...,S037673882200343X,Irish Valerie Maggay,"R&D Center for Membrane Technology, Chung Yuan...",121,"Versatile, facile, easy, and economical, these..."
1,Polysulfone (PSf) membrane is amongst the most...,Strategy to prepare skin-free and macrovoid-fr...,S037673882200343X,Irish Valerie Maggay,"R&D Center for Membrane Technology, Chung Yuan...",299,Polysulfone (PSf) membrane is amongst the most...
2,As a result of the raising interest to improve...,Strategy to prepare skin-free and macrovoid-fr...,S037673882200343X,Irish Valerie Maggay,"R&D Center for Membrane Technology, Chung Yuan...",106,As a result of the raising interest to improve...
3,Standard NIPS process yield skinned asymmetric...,Strategy to prepare skin-free and macrovoid-fr...,S037673882200343X,Irish Valerie Maggay,"R&D Center for Membrane Technology, Chung Yuan...",264,Standard NIPS process yield skinned asymmetric...
4,Although adding solvent to the coagulation bat...,Strategy to prepare skin-free and macrovoid-fr...,S037673882200343X,Irish Valerie Maggay,"R&D Center for Membrane Technology, Chung Yuan...",217,Although adding solvent to the coagulation bat...
...,...,...,...,...,...,...,...
2114,It was observed that at a higher b-CD concentr...,Tight ultrafiltration: Layer deposition of Tri...,S2213343720300816,John Michael Tesha,State Key Laboratory of Separation Membranes a...,139,It was observed that at a higher b-CD concentr...
2115,Tight UF membranes were successfully prepared ...,Tight ultrafiltration: Layer deposition of Tri...,S2213343720300816,John Michael Tesha,State Key Laboratory of Separation Membranes a...,171,Tight UF membranes were successfully prepared ...
2116,John Michael Tesha: Writing - review & editing...,Tight ultrafiltration: Layer deposition of Tri...,S2213343720300816,John Michael Tesha,State Key Laboratory of Separation Membranes a...,58,John Michael Tesha: Writing - review & editing...
2117,The authors declare that they have no known co...,Tight ultrafiltration: Layer deposition of Tri...,S2213343720300816,John Michael Tesha,State Key Laboratory of Separation Membranes a...,26,The authors declare that they have no known co...


In [71]:
test_paper_titles = papers.get_all_titles()
test_paper_piis = papers.piis

In [76]:
example_df

,paragraph,label,word_count,split
0,"Versatile, facile, easy, and economical, these...",n,121,"Versatile, facile, easy, and economical, these..."
1,Polysulfone (PSf) membrane is amongst the most...,n,299,Polysulfone (PSf) membrane is amongst the most...
2,As a result of the raising interest to improve...,n,106,As a result of the raising interest to improve...
3,Standard NIPS process yield skinned asymmetric...,n,264,Standard NIPS process yield skinned asymmetric...
4,Although adding solvent to the coagulation bat...,n,217,Although adding solvent to the coagulation bat...
...,...,...,...,...
482,"Yi Yu: Conceptualization, Methodology, Investi...",n,39,"Yi Yu: Conceptualization, Methodology, Investi..."
483,The authors declare that they have no known co...,n,26,The authors declare that they have no known co...
484,This work is financially supported by National...,n,49,This work is financially supported by National...
485,The following is the Supplementary data to thi...,n,13,The following is the Supplementary data to thi...


In [102]:
test_papers

,paragraph,title,pii,author,organization,word_count,split
0,"Versatile, facile, easy, and economical, these...",Strategy to prepare skin-free and macrovoid-fr...,S037673882200343X,Irish Valerie Maggay,"R&D Center for Membrane Technology, Chung Yuan...",121,"Versatile, facile, easy, and economical, these..."
1,Polysulfone (PSf) membrane is amongst the most...,Strategy to prepare skin-free and macrovoid-fr...,S037673882200343X,Irish Valerie Maggay,"R&D Center for Membrane Technology, Chung Yuan...",299,Polysulfone (PSf) membrane is amongst the most...
2,As a result of the raising interest to improve...,Strategy to prepare skin-free and macrovoid-fr...,S037673882200343X,Irish Valerie Maggay,"R&D Center for Membrane Technology, Chung Yuan...",106,As a result of the raising interest to improve...
3,Standard NIPS process yield skinned asymmetric...,Strategy to prepare skin-free and macrovoid-fr...,S037673882200343X,Irish Valerie Maggay,"R&D Center for Membrane Technology, Chung Yuan...",264,Standard NIPS process yield skinned asymmetric...
4,Although adding solvent to the coagulation bat...,Strategy to prepare skin-free and macrovoid-fr...,S037673882200343X,Irish Valerie Maggay,"R&D Center for Membrane Technology, Chung Yuan...",217,Although adding solvent to the coagulation bat...
...,...,...,...,...,...,...,...
2114,It was observed that at a higher b-CD concentr...,Tight ultrafiltration: Layer deposition of Tri...,S2213343720300816,John Michael Tesha,State Key Laboratory of Separation Membranes a...,139,It was observed that at a higher b-CD concentr...
2115,Tight UF membranes were successfully prepared ...,Tight ultrafiltration: Layer deposition of Tri...,S2213343720300816,John Michael Tesha,State Key Laboratory of Separation Membranes a...,171,Tight UF membranes were successfully prepared ...
2116,John Michael Tesha: Writing - review & editing...,Tight ultrafiltration: Layer deposition of Tri...,S2213343720300816,John Michael Tesha,State Key Laboratory of Separation Membranes a...,58,John Michael Tesha: Writing - review & editing...
2117,The authors declare that they have no known co...,Tight ultrafiltration: Layer deposition of Tri...,S2213343720300816,John Michael Tesha,State Key Laboratory of Separation Membranes a...,26,The authors declare that they have no known co...


In [104]:
co = cohere.Client(cohere_apiKey) 
response = co.classify( 
  model='large', 
  inputs=[i for i in test_papers['split']][:32], 
  examples = [Example(example_df['split'][i], example_df['label'][i]) for i in range(len(example_df))][:50])

In [47]:
# append the results to the test_papers dataframe
test_papers['label'] = [i.prediction for i in response.classifications]
test_papers['confidence'] = [i.confidence for i in response.classifications]
test_papers.to_csv('test_data_labeled.csv')

In [48]:
# save the rows with label y and confidence > 0.8 and drop index
test_papers = test_papers[test_papers['label'] == 'y']
test_papers = test_papers[test_papers['confidence'] > 0.8]
test_papers = test_papers.reset_index(drop = True)
test_papers.to_csv('test_data_labeled_confidence.csv')

# Information Extraction

In [8]:
test_papers = pd.read_csv('test_data_labeled.csv')
#test_papers = pd.read_csv('Example_data_labeled_updated.csv')
# if label is y, then join all the adjacent paragraphs if their labels are y
for i in range(len(test_papers)):
    if test_papers['label'][i] == 'y':
        j = 0
        # if the next row is y, then join the current row with the next row
        while test_papers['label'][i+j+1] == 'y':
            test_papers['paragraph'][i] = test_papers['paragraph'][i] + ' ' + test_papers['paragraph'][i+j+1]
            test_papers['label'][i] = 'y'
            # change the label of the next row to n
            test_papers['label'][i+j+1] = 'n'
            j += 1
test_papers

C:\Users\wang9\AppData\Roaming\Python\Python37\site-packages\ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == "__main__":
C:\Users\wang9\AppData\Roaming\Python\Python37\site-packages\ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # Remove the CWD from sys.path while we load stuff.
C:\Users\wang9\AppData\Roaming\Python\Python37\site-packages\ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.

,Unnamed: 0,paragraph,word_count,split,label,confidence
0,0,"Today, organic solvent nanofiltration (OSN) ha...",151,"Today, organic solvent nanofiltration (OSN) ha...",n,0.999999
1,1,The solvent resistant nanofiltration membrane ...,92,The solvent resistant nanofiltration membrane ...,n,1.000000
2,2,Polymeric membranes are usually inexpensive ma...,76,Polymeric membranes are usually inexpensive ma...,n,1.000000
3,3,"Polysulfone (PSf), a traditional aromatic poly...",146,"Polysulfone (PSf), a traditional aromatic poly...",n,0.999999
4,4,Cross-linking reactions are considered as cons...,123,Cross-linking reactions are considered as cons...,n,0.999835
...,...,...,...,...,...,...
1614,1614,A detailed comparison of permeability-selectiv...,103,A detailed comparison of permeability-selectiv...,n,1.000000
1615,1615,PSf membranes with improved antifouling and he...,125,PSf membranes with improved antifouling and he...,n,1.000000
1616,1616,The following is the supplementary data relate...,18,The following is the supplementary data relate...,n,1.000000
1617,1617,Supplementary data to this article can be foun...,11,Supplementary data to this article can be foun...,n,1.000000


In [4]:
test_papers = pd.read_csv('test_data_labeled_joined.csv')
# drop the rows with label n
test_papers = test_papers[test_papers['label'] == 'y']
test_papers = test_papers.reset_index(drop = True)
paragraphs = test_papers['paragraph']
paragraphs

0     The preparation of chloromethylated polysulfon...
1     PVC resin (Mw = 81,000 g*mol-1) was obtained f...
2     Polysulfone (PSf, MW 80 kDa) was supplied by D...
3     The PSf-sg-AB12C4 membrane (including dense me...
4     PSF was purchased from Arvand Co., Iran. N-dim...
                            ...                        
63    A series of allyl polysulfone-graft-poly(dopam...
64    A series of ultrafiltration (UF) membranes con...
65    The porous support UF membranes (PSF/SMA) were...
66    The PSF/SMA support membrane was first saturat...
67    Polysulfone (PSf, S6010) was bought from BASF ...
Name: paragraph, Length: 68, dtype: object

In [5]:
co = cohere.Client(cohere_apiKey)
json_list = []
for i in range(len(paragraphs)):
    paragraph = paragraphs[i]
    print('Paragraph: {}'.format(paragraphs[i]))
    while True: 
        try: 
            response = co.generate( 
                model='xlarge', 
                prompt='Paragraph: N,N-methyl-pyrrolidone (NMP, >=99.0% purity) purchased from Shanghai Taitan, Ltd. (Shanghai, China) was used as solvent and deionized water was used as non-solvent in the NIPS processes. Polysulfone (PSF, S6010) was purchased from Dongguan Haosheng Plastic Material Company and dried under vacuum at 60 degC for 12 h before use. Polyvinylpyrrolidone (PVP, >98%)) and 2-methylimidazole (2MIM, Analytical Grade) were purchased from Aladdin, Shanghai. Zn (NO3) 2*6H2O was purchased from Sinopharm Chemical Reagent Company. Congo Red (CR), Methyl Orange (MO) and Methylene Blue (MBe) were purchased from Aladdin (Shanghai) Reagent Company, Shanghai Maclean Biochemical Technology Company and Sinopharm Chemical Reagent company respectively. Polyethylene terephthalate (PET) NWF were cut into 16cm x 16 cm piece and then were poured and completely wet by a 30 g/L Zn (NO3) 2*6H2O aqueous solution. After removal of excess liquid, the solution loading weight was measured for the calculation of Zn (NO3)2 loading. The Zn (II)-loaded NWF were then dried in oven at 30 degC for 0.5 h before use. The fully de-foamed cast solution was then cast onto the Zn (II)-loaded NWF at a thickness of 250 mm using a coating machine. The coated film was then clamped into two poly-tetrafluoro-ethylene (PTFE) frames with inner diameters of 14cm x 14 cm, and then immersed into a 50 degC de-ionized (DI) water bath for a certain time for phase separation. During the phase separation process, 2MIM from the casting solution and Zn2+ from NWF were continuously ligated to form ZIF-8 NPs in the PSF matrix. In the end, the formed mixed matrix membrane was dried at 30 degC in an oven for 40 min and then sealed and stored in DI-water for testing. 18 g of polysulfone (PSF), 6 g of ZIF-8 particles, and 2 g of polyvinylpyrrolidone (PVP) were added to N-Methylpyrrolidone (NMP), with a total mass of 100 g. By continuous stirring in an oil bath at 50 degC (200 rpm), a casting suspension was obtained and then stand still overnight to be fully de-foamed. The fully de-foamed suspension was then cast onto the pristine PET NWF at a thickness of 250 mm using a coating machine. The coated film was then clamped into two poly-tetrafluoro-ethylene (PTFE) frames with inner diameters of 14cm x 14 cm, and then immersed into a 50 degC de-ionized (DI) water bath for a certain time for phase separation. In the end, the formed mixed matrix membrane was dried at 30 degC in an oven for 40 min and then sealed and stored in DI-water for testing.\nDictionary: {\"polymer\": \"Polysulfone\", \"solvent\": \"N,N-methyl-pyrrolidone\", \"non-solvent\": \"deionized water\", \"additives\": \"ZIF-8 particles\", \"polymer concentration\": 18, \"amount of polymer\": 18, \"amount of polymer unit\": \"g\", \"amount of solvent\": 74, \"amount of solvent unit\": \"g\", \"synthesis temperature\": 50, \"synthesis temperature unit\": \"C\", \"synthesis time\": \"NA\", \"synthesis time unit\": \"NA\"}\n--\nParagraph: The polymer solution preparation technique was described in [40]. Polymer solution (20 wt% PSf, 10 wt% PEG-400 and 70 wt% DMAc) was applied on a glass plate using a casting knife with a gap height of 250 um at the room temperature. Distilled water and PAA aqueous solutions (0.4-2.0 wt%) were used as the coagulation bath during membrane preparation via NIPS. PAA concentrations in the coagulation bath and corresponding membrane abbreviations are presented in Table 2. Membranes were kept in distilled water at least for 24 h to remove the residual solvent. The time of membrane formation in NIPS was determined as the time from the moment of immersion of a nascent polymer film into the coagulation bath to the moment, when the precipitated membrane completely separates from the glass plate. Membrane formation time was determined for different coagulation bath compositions at T = 25 degC (Table 2).\nDictionary: {\"polymer\": \"PSf\", \"solvent\": \"DMAc\", \"non-solvent\": \"Distilled water and PAA aqueous solutions\", \"additives\": \"NA\", \"polymer concentration\": \"20\", \"amount of polymer\": \"NA\", \"amount of polymer unit\": \"NA\", \"amount of solvent\": \"NA\", \"amount of solvent unit\": \"NA\", \"synthesis temperature\": 25, \"synthesis temperature unit\": \"C\", \"synthesis time\": 24, \"synthesis time unit\": \"h\"}\n--\nParagraph:'+paragraph+'\nDictionary:',
                max_tokens=150, 
                temperature=0, 
                k=0, 
                p=0.75, 
                frequency_penalty=0, 
                presence_penalty=0, 
                stop_sequences=["--"], 
                return_likelihoods='NONE')
            object = response.generations[0].text
            object = object[object.find('{'):object.rfind('}')+1]
            print('Prediction: {}'.format(object)) 
            print('-----------------------------------------------------------------------------')
            json_list.append(object)
        except:
            while True:
                try:
                    response_sum = co.generate( 
                        model='command-xlarge-20221108', 
                        prompt='Summarize the following paragraph in concise language so that all key information (chemical names and numerical values) are kept and word count are reduced by at least half:\nparagraph: '+paragraph+ '\nSummary: ', 
                        max_tokens=500, 
                        temperature=0.6, 
                        k=0, 
                        p=1, 
                        frequency_penalty=0, 
                        presence_penalty=0, 
                        stop_sequences=["--"], 
                        return_likelihoods='NONE')
                    paragraph = (response_sum.generations[0].text)
                except: 
                    paragraph = paragraph[:int(len(paragraph)*0.9)]
                    continue
                break
            continue
        break
    

Paragraph: The preparation of chloromethylated polysulfone (CMPSf) was done based on the literature [28] (Scheme 1). Accordingly, PSf (10 g) was introduced into a flask, which was equipped with a reflux condenser, containing chloroform (500 mL), followed by stirring in order to form a homogenous solution. In the next step, paraformaldehyde (6.79 g) and trimethylchlorosilane (24.7 g) were gradually introduced into the above-mentioned PSf solution. Next, stannic chloride (0.6 mL) was slowly introduced into the as-fabricated mixture solution, followed by heating at 55 degC for 2 days. In the final step, the preparation of CMPSf was done through pouring the as-fabricated mixture solution into a methanol bath, followed via drying at 40 degC for 48 h at a vacuum oven.(Schemes 2 and 3). Fumed Silica (0.5 g) was added into 100 mL of Ethanol/deionized water (9:1) solution under sonication, followed via the addition of 1 mL of APTES under stirring at 60 degC for 24 h so that amino-functionalized

Prediction: {"polymer": "PSf", "solvent": "DMAc", "non-solvent": "Distilled water and PAA aqueous solutions", "additives": "NA", "polymer concentration": "20", "amount of polymer": "NA", "amount of polymer unit": "NA", "amount of solvent": "NA", "amount of solvent unit": "NA", "synthesis temperature": 25, "synthesis temperature unit": "C", "synthesis time": 24, "synthesis time unit": "h"}
-----------------------------------------------------------------------------
Paragraph: PVC resin (Mw = 81,000 g*mol-1) was obtained from Qilu Petrochemical Company, China. Sulfonated polysulfone (SPSF) with different sulfonation degree (SD) of 20% (Mw = 68,000 g*mol-1), 30% (Mw = 65,000 g*mol-1) and 50% (Mw = 63,000 g*mol-1), purchased from Tianjin Yanjin Technology Co. Ltd., China, were used as the membrane additives. Polyethylene glycols (PEGs, Mw = 400, 10 k, 20 k, 35 k, and 100 k Da) and polyethylene oxide (PEO, Mw = 600 k Da) were purchased from Sigma-Aldrich Reagent Co., USA. PEG 400 was used 

Prediction: {"polymer": "Polysulfone", "solvent": "N,N-methyl-pyrrolidone", "non-solvent": "deionized water", "additives": "PEI-SiO2 nanoparticles", "polymer concentration": "20", "amount of polymer": "NA", "amount of polymer unit": "NA", "amount of solvent": "NA", "amount of solvent unit": "NA", "synthesis temperature": 25, "synthesis temperature unit": "C", "synthesis time": "NA", "synthesis time unit": "NA"}
-----------------------------------------------------------------------------
Paragraph: PSF (Udel(r)P-3500, Mw ~ 78000 Da) was supplied by Solvay Co., Ltd. (Brussels, Belgium). SMA (anh) (725, anhydride content~25, molecular weight ~ 140000 Da) was purchased from Jiaxing Huawen Chemical Co., Ltd. (Jiaxing, China), and SMA was prepared according to the literature [17]. Other reagents were bought from Sinopharm Chemical Reagent Co., Ltd. (Shanghai, China). The non-woven PET (thickness~140 mm) was obtained from Suzhou Tuopusi Environmental Protection Technology Co., Ltd. (Suzhou, 

Prediction: {"polymer": "Polysulfone", "solvent": "N,N-dimethylacetamide", "non-solvent": "Deionized water", "additives": "NA", "polymer concentration": "18.0", "amount of polymer": "NA", "amount of polymer unit": "NA", "amount of solvent": "NA", "amount of solvent unit": "NA", "synthesis temperature": "25", "synthesis temperature unit": "C", "synthesis time": "NA", "synthesis time unit": "NA"}
-----------------------------------------------------------------------------
Paragraph: The monomer Z and the polymer PESSB were prepared according to the method of Zhang et al[50]. According to our previously reported work, the sulfobetaine-containing polymer PESSB was synthesized via polycondensation and quaternization. We designed and synthesized a series of copolymers containing diblocks of PESTA and PSF for post-treatment of hydrophilization modification. A facile one-pot method was used to prepare copolymer PSF-PESTA. By varying the molar ratio of diphenol monomers, we were able to prepar

Prediction: {"solvent": "NMP", "non-solvent": "DI water", "additives": "NaOH", "TiO2", "CoCl2", "SiO2", "NH3-N", "C6H5OH", "C6H4O2", "H2C2O4", "PRW", "Cilacap", "Central Java", "PT. Pertamina (persero)", "PT. Pertamina (persero)", "Cilacap", "Central Java", "Indonesia"}
-----------------------------------------------------------------------------
Paragraph: SPSf with a sulfonation degree of 5% obtained from Tianjin Yanjin Technology Co., Ltd., (China) was chosen as the membrane matrix. Troger's base (TB) polymer (see Scheme 1) was synthesized from dimethoxymethane (98.0%), 4, 4'-diamino-3, 3'-dimethyl-biphenyl (98%), trifluoroacetic acid (TFA, 99.0%) which were purchased from Shanghai Aladdin Bio-Chem Technology Co., Ltd., (China). Ammonia solution (NH4OH, 99.0%), trichloromethane (CHCl3, >99.0%), and methanol (CH3OH, 99.7%) purchased from Sinopharm Chemical Reagent Co., Ltd., (China) were used during the precipitation and washing process of Troger's base (TB) polymer. N-methyl-2-pyrro

...

Prediction: 
-----------------------------------------------------------------------------
Paragraph: Sulfonated polysulfone (SPSF, IEC = 1.91 meq/g, [e] > 0.6 dL/g) with a degree of sulfonation (DS) of 50% was synthesized in laboratory. Polyethersulfone (PES) membranes with different Molecular Weight Cut-Off (MWCO, 10 kDa, 20 kDa and 50 kDa) were purchased from Beijing Separate Equipment Co., Ltd. N,N-Dimethylacetamide (DMAc, 99.0%), sodium sulfate (>=99.0%), magnesium sulfate (>=99.0%), sodium chloride (>=99.0%) and magnesium chloride (>=99.0%) were purchased from Sinopharm Chemical Reagent Co., Ltd., China. Polyethylene glycol (PEG) with different molecular weights (400, 600, 1000, 2000, 4000 Da) were purchased from Aladdin Reagent Co., Ltd., China. Deionized (DI) water (PH  6.5) was used in all experiments. The NFMs was fabricated as follows: First, a SPSF solution in DMAc (5 wt%) was dropwise added to DI water under vigorous stirring and ultrasound to form homogeneous dispersion o

Prediction: {"polymer": "P4VP-PSF-P4VP", "polymer concentration": "0.1", "amount of polymer": "0.1", "amount of polymer unit": "0.1", "amount of solvent": "10", "amount of solvent unit": "10", "synthesis temperature": 80, "synthesis temperature unit": "C", "synthesis time": 24, "synthesis time unit": "h"}
-----------------------------------------------------------------------------
Paragraph: Pd (II) stock solution with a concentration of 1000 mg/L was prepared by dissolving palladium (II) chloride in 2 mol/L HCl (Suryavanshi et al., 2015). The batch adsorption experiment for a membrane was done by adding 0.05 g of the membrane into a 250 mL conical flask containing 100 mL of Pd (II) solution (60 mg/L). Generally, the pH of solution was maintained at 2 according to the literature (Zhou et al., 2010). Then, the solution was shaken on a shaker at 25 degC for 24 h, and the Pd (II) concentration of the solution was detected by the ICP-OES. For studying the effect of pH on the adsorption ca

Prediction: {"polymer": "PSf", "solvent": "DMAc", "non-solvent": "Deionized water", "additives": "CuI superstructure nanosheets", "polymer concentration": "16", "amount of polymer": "NA", "amount of polymer unit": "NA", "amount of solvent": "NA", "amount of solvent unit": "NA", "synthesis temperature": 70, "synthesis temperature unit": "C", "synthesis time": 12, "synthesis time unit": "h"}
-----------------------------------------------------------------------------
Paragraph: Udel P-1700 polysulfone was purchased from Solvay Advanced Polymer LLC and used as polymer. N-Methyl-2-Pyrrolidon (NMP) and polyethylene glycol (PEG; MW = 20,000 g/gmol) were purchased from Daejung and Sigma, respectively. The pristine detonation ND nanoparticles with a phase purity higher than 95% were procured from Nabond technology Co., Ltd., China. The average diameter and specific surface area of ND nanoparticles were about 5 nm and 282.8 m2 g-1, respectively. Vinyltrimethoxysilane (VTS) with the purity of 9

Prediction: {"polymer": "PSf", "solvent": "NMP", "non-solvent": "Toluene", "additives": "NA", "polymer concentration": "20", "amount of polymer": "NA", "amount of polymer unit": "NA", "amount of solvent": "NA", "amount of solvent unit": "NA", "synthesis temperature": "NA", "synthesis temperature unit": "NA", "synthesis time": "NA", "synthesis time unit": "NA"}
-----------------------------------------------------------------------------
Paragraph: A series of allyl polysulfone-graft-poly(dopamine methacrylamide) membranes (AP-DM# membranes, where # is the weight ratio of DMA to APSf) were prepared by an in situ process composed of grafting-through polymerization followed by a non-solvent induced phase separation method [28,29]. For the preparation of AP-DM1 membrane, APSf (1.00 g), AIBN (0.10 g), PSf (1.00 g), and DMA (1.00 g) were dissolved in 10.3 mL of NMP in a 100 mL dried Schlenk flask containing a magnetic stir bar. PSf is the commercial polysulfone, and it was intentionally used

Prediction: {"polymer": "Polysulfone", "solvent": "N,N-methyl-pyrrolidone", "non-solvent": "deionized water", "additives": "AIBN", "polymer concentration": "1", "amount of polymer": "1", "amount of polymer unit": "NA", "amount of solvent": "NA", "amount of solvent unit": "NA", "synthesis temperature": 80, "synthesis temperature unit": "C", "synthesis time": 6, "synthesis time unit": "h"}
-----------------------------------------------------------------------------
Paragraph: A series of ultrafiltration (UF) membranes consisting of allyl polysulfone-graft-poly(dopamine methacrylamide) (AP-DM# membranes, where # indicates the feed ratio of APSf to dopamine methacrylamide (DMA)) were fabricated through the following four steps (Scheme 2(a)): (1) preparing dope solutions for the polymerization of APSf and DMA containing vinyl moieties (grafting-through), (2) casting the resulting solutions onto the PET non-woven fabric, (3) immersing the casted films for 24 h in a DI water bath for the for

In [6]:
json_list

['{"polymer": "PSf", "solvent": "DMAc", "non-solvent": "Distilled water and PAA aqueous solutions", "additives": "NA", "polymer concentration": "20", "amount of polymer": "NA", "amount of polymer unit": "NA", "amount of solvent": "NA", "amount of solvent unit": "NA", "synthesis temperature": 25, "synthesis temperature unit": "C", "synthesis time": 24, "synthesis time unit": "h"}',
 '{"solvent": "DMAc", "non-solvent": "DI water", "additives": "SPSF", "polymer concentration": "20", "amount of polymer": "NA", "amount of polymer unit": "NA", "amount of solvent": "NA", "amount of solvent unit": "NA", "synthesis temperature": 60, "synthesis temperature unit": "C", "synthesis time": 8, "synthesis time unit": "h"}',
 '{"polymer": "Polysulfone", "solvent": "Chlorotrimethylsilane", "non-solvent": "1,3,5-trioxane", "additives": "SnCl4", "polymer concentration": "4", "amount of polymer": "4", "amount of polymer unit": "g", "amount of solvent": "18.2", "amount of solvent unit": "mL", "synthesis tem

In [7]:
#save the json list to a file
with open('json_list.txt', 'w') as f:
    for item in json_list:
        f.write("%s\n" % item)

In [13]:
myobject = json.loads(json_list[3])
myobject

{'polymer': 'PSf',
 'solvent': 'DMAc',
 'non-solvent': 'Distilled water and PAA aqueous solutions',
 'additives': 'NA',
 'polymer concentration': '20',
 'amount of polymer': 'NA',
 'amount of polymer unit': 'NA',
 'amount of solvent': 'NA',
 'amount of solvent unit': 'NA',
 'synthesis temperature': 60,
 'synthesis temperature unit': 'C',
 'synthesis time': 6,
 'synthesis time unit': 'h'}

In [23]:
response.generations[0].text

'\n\nThe type of polymer is polysulfone, the type of solvent is water, the type of additives is AB12C4, the polymer concentration is 10%, the amount of polymer is 5 g, the amount of solvent is 25 g, the synthesizing temperature is 60 degrees Celsius, the synthesize time is 6 hours.'